Goal of this notebook is to understand how to call cphf.solve_nos1(fvind, mo_energy, mo_occ, h1,max_cycle=20, tol=1e-9, hermi=False)  (better the shape of the arrays passed as arguments)

In [8]:
import pyscf
import pyscf.qmmm
from pyscf import gto, scf
import numpy as np
import matplotlib.pyplot as plt
import inspect
#from pyscf.geomopt.berny_solver import optimize
#from pyscf.grad import rhf as grhf
from pyscf.hessian import rhf as hrhf
from pyscf import lib
import inspect
from functools import reduce
from pyscf.scf import cphf
from pyscf.prop.polarizability.rhf import Polarizability
angstrom = 1 / 0.52917721067
#from pyscf.scf._response_functions import _gen_rhf_response 

In [9]:
def DeltaV(mol,dL):
    mol.set_rinv_orig_(mol.atom_coords()[0])
    dV=mol.intor('int1e_rinv')*dL[0]
    mol.set_rinv_orig_(mol.atom_coords()[1])
    dV+=mol.intor('int1e_rinv')*dL[1]
    return -dV.reshape((1,dV.shape[0],dV.shape[1]))

def fc(calc,deltaZ):
    mf = pyscf.qmmm.mm_charge(calc, calc.mol.atom_coords(), deltaZ)  # now is add_mm_charge
    class NoSelfQMMM(mf.__class__):
        def energy_nuc(self):
            q = self.mol.atom_charges().astype(np.float).copy()
            q1 =q+ np.asarray(deltaZ) 
            return self.mol.energy_nuc(q1)
    return(NoSelfQMMM(mf,mf.mm_mol))

In [118]:
import time
from functools import reduce
import numpy
from pyscf import lib
from pyscf.lib import logger
from pyscf.scf import cphf
from pyscf.prop.nmr import rhf as rhf_nmr
#from pyscf.scf import _gen_rhf_response

def dipole(mf):
    return mf.dip_moment(mf.mol, mf.make_rdm1())


def polarizability(polobj, with_cphf=True):
    
    log = logger.new_logger(polobj)
    mf = polobj._scf
    mol = mf.mol
    mo_energy = mf.mo_energy
    mo_coeff = mf.mo_coeff
    mo_occ = mf.mo_occ
    occidx = mo_occ > 0
    orbo = mo_coeff[:, occidx]
    orbv = mo_coeff[:,~occidx]

    charges = mol.atom_charges()
    coords  = mol.atom_coords()
    charge_center = numpy.einsum('i,ix->x', charges, coords) / charges.sum()
    int_r=DeltaV(mol,[.001,-.001])
    h1 = lib.einsum('xpq,pi,qj->xij', int_r, mo_coeff.conj(), orbo) #going to molecular orbitals?
    print(h1.shape, "shape (1, n ao, nocc)" )
    #print(mo_energy, mo_occ,h1)
    s1 = numpy.zeros_like(h1)
    vind = polobj.gen_vind(mf, mo_coeff, mo_occ)
    if with_cphf:
        mo1 = cphf.solve(vind, mo_energy, mo_occ, h1, s1, polobj.max_cycle_cphf, polobj.conv_tol,verbose=log)[0]
    else:
        mo1 = rhf_nmr._solve_mo1_uncoupled(mo_energy, mo_occ, h1, s1)[0]
    #e2 = numpy.einsum('xpi,ypi->xy', h1, mo1)
    # *-1 from the definition of dipole moment. *2 for double occupancy
    #e2 = (e2 + e2.T) * -2
    #return e2
    return mo1

def gen_vind(self, mf, mo_coeff, mo_occ):  #polarizability
        '''Induced potential'''
        vresp = mf.gen_response(hermi=1)
        occidx = mo_occ > 0
        orbo = mo_coeff[:, occidx]
        nocc = orbo.shape[1]
        nao, nmo = mo_coeff.shape
        def vind(mo1):
            dm1 = lib.einsum('xai,pa,qi->xpq', mo1.reshape(-1,nmo,nocc), mo_coeff,orbo.conj())
            dm1 = (dm1 + dm1.transpose(0,2,1).conj()) * 2
            v1mo = lib.einsum('xpq,pi,qj->xij', vresp(dm1), mo_coeff.conj(), orbo)
            return v1mo.ravel()
        return vind

In [179]:
mol = gto.Mole(atom='H 0 0 0; H 0 0 1.2', unit="Bohr",basis="STO-3G")
#mol=gto.Mole(atom="C 0 0 0 ;O 0 0 2.", unit="Bohr", basis= "3-21G")
mf = scf.RHF(mol)
e=mf.scf()

Initialize <pyscf.gto.mole.Mole object at 0x7fc2c09c45d0> in <pyscf.scf.hf.RHF object at 0x7fc2c0a0a6d0>


converged SCF energy = -1.11033388268018


In [180]:
plo=Polarizability(mf) #polariz obj

In [181]:
pa=polarizability(plo)

(1, 2, 1) shape (1, n ao, nocc)


In [182]:
hf1=fc(mf,[.001,-0.001])
hf1.scf()

converged SCF energy = -1.1103355584584


-1.1103355584583992

In [183]:
C1= hf1.mo_coeff
C1

array([[ 0.53859761, -1.35712572],
       [ 0.53702883,  1.35774726]])

In [196]:
C=np.flip(mf.mo_coeff.copy(),axis=0)

In [197]:
C

array([[ 0.53781331, -1.35743672],
       [ 0.53781331,  1.35743672]])

In [198]:
C1-C

array([[ 0.0007843 ,  0.000311  ],
       [-0.00078448,  0.00031055]])

In [201]:
nao=mol.nao
nocc=mol.nelectron//2 #RHF
U=np.zeros((nao,nao))
U[:,:nocc]=pa[0,:,:nocc]
U=U.T-U
U

array([[ 0.        ,  0.00057807],
       [-0.00057807,  0.        ]])

In [202]:
C@U-(C1-C)

array([[ 3.96287780e-07, -1.05195689e-07],
       [-2.16707694e-07,  3.48063087e-07]])

In [203]:
C+C@U,C1

(array([[ 0.53859801, -1.35712582],
        [ 0.53702861,  1.35774761]]),
 array([[ 0.53859761, -1.35712572],
        [ 0.53702883,  1.35774726]]))

In [244]:
mol=gto.Mole(atom="C 0 0 0 ;O 0 0 2.", unit="Bohr", basis= "def2-QZVPP")
mf = scf.RHF(mol)
e=mf.scf()
C=mf.mo_coeff
P=mf.make_rdm1()

Initialize <pyscf.gto.mole.Mole object at 0x7fc2c0a77b90> in <pyscf.scf.hf.RHF object at 0x7fc2c0a77a90>


converged SCF energy = -112.786331647844


In [245]:
plo=Polarizability(mf)

In [246]:
pa=polarizability(plo)
hf1=fc(mf,[.001,-0.001])
hf1.scf()
P1=hf1.make_rdm1()

(1, 114, 7) shape (1, n ao, nocc)
converged SCF energy = -112.778763010131


In [247]:
nao=mol.nao
nocc=mol.nelectron//2 #RHF
U=np.zeros((nao,nao))
U[:,:nocc]=pa[0,:,:nocc]
U=U-U.T
O=np.diag(mf.mo_occ)

In [248]:
dP_app=C@(U@O-O@U)@C.T

In [249]:
np.max(P1-P)

0.0005164201220738684

In [250]:
np.max(dP_app)

0.0005162556296109519

In [251]:
np.max(dP_app-P1+P)

3.139931471496382e-07